In [1]:
import torch
import torch.nn as nn
from pysmt.shortcuts import Symbol, And, Equals, Real, GT, Max, Plus, get_model
from pysmt.typing import REAL

In [2]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()             
        #self.linear0 = nn.Linear(3, 3)
        self.linear1 = nn.Linear(3, 1)
        #self.linear_layers = [self.linear0, self.linear1]
        self.linear_layers = [self.linear1]
        self.relu = nn.ReLU()
        
    def forward(self, x):   
        #x = self.linear0(x)
        #x = self.relu(x)  
        
        x = self.linear1(x)
        x = self.relu(x) 
        
        return x[0]

In [3]:
def train_backpropagation(model, all_data, labels, criterion, optimizer, num_epochs):
    model.train()
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        num_correct = 0
    
        for (data, label) in zip(all_data, labels): 
            optimizer.zero_grad()
        
            output = model(data)
            prediction = 1 if output > 0 else 0

            loss = criterion(output, label)
            num_correct += (label == prediction)
        
            running_loss += loss.item()
            loss.backward()
            optimizer.step()

        loss = running_loss/len(labels)
        accuracy = num_correct/len(labels)
        print("epoch: " + str(epoch) + ", accuracy: " + str(accuracy.item()) + ", loss: " + str(loss))
        
        if (accuracy == 1.0):
            return

In [4]:
def train_smt(model, all_data, labels):
    
    layers_weights = []
    num_layers = len(model.linear_layers)
    for i in range(num_layers):
        num_rows = len(model.linear_layers[i].weight.data)
        num_cols = len(model.linear_layers[i].weight.data[0])
        layer_weights = []
        for j in range(num_rows):
            weights = []
            for k in range(num_cols):
                weights.append(Symbol("weight_" + str(j) + "_" + str(k), REAL))
            weights.append(Symbol(str("bias_" + str(j)), REAL))
            layer_weights.append(weights)
        layers_weights.append(layer_weights)

    equations = []
    for (data, label) in zip(all_data, labels):
        
        for i in range(num_rows):
            calculations = []
            for j in range(num_cols):
                x = data[j].item()
                calculation = layers_weights[0][i][j] * Real(x)
                calculations.append(calculation)
            calculations.append(layers_weights[0][i][num_cols])
            calculation = Plus(calculations)
            
        relu = Max(calculation, Real(0.0))
    
        if label.item() == 0.0:
            equation = Equals(relu, Real(0.0))
        else:
            equation = GT(relu, Real(0.0))
            
        equations.append(equation)

    formula = And(equations)
    solution = get_model(formula)
    
    if solution:
        print(solution)
        
        for i in range(num_layers):
            num_rows = len(model.linear_layers[i].weight.data)
            num_cols = len(model.linear_layers[i].weight.data[0])
            for j in range(num_rows):
                for k in range(num_cols):
                    weight = layers_weights[i][j][k]
                    weight_solution = solution[weight].constant_value()
                    model.linear_layers[i].weight.data[j][k] = float(weight_solution)
    
                bias = layers_weights[i][j][num_rows]
                bias_solution = solution[bias].constant_value()
                model.linear_layers[i].bias.data[i] = float(bias_solution)

    else:
        print("No solution found")

In [5]:
def test(model, all_data, labels, criterion): 
    running_loss = 0.0
    num_correct = 0
    
    for (data, label) in zip(all_data, labels): 
        output = model(data)
        prediction = 1 if output > 0 else 0

        loss = criterion(output, label)
        num_correct += (label == prediction)
        
        running_loss += loss.item()

    loss = running_loss/len(labels)
    accuracy = num_correct/len(labels)
    
    print("testing accuracy: " + str(accuracy.item()) + " and loss: " + str(loss))

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Model()
model.to(device)

points = torch.tensor([[-1.0, -1.0, -1.0], [1.0, -1.0, -1.0], [-1.0, 1.0, -1.0], [-1.0, -1.0, 1.0],
                       [1.0, 1.0, 1.0], [-1.0, 1.0, 1.0], [1.0, -1.0, 1.0], [1.0, 1.0, -1.0],])

labels = torch.tensor([0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0], requires_grad=True)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)
num_epochs = 200

In [7]:
train_backpropagation(model, points, labels, criterion, optimizer, num_epochs)
test(model, points, labels, criterion)

epoch: 0, accuracy: 0.625, loss: 0.6600987166166306
epoch: 1, accuracy: 0.625, loss: 0.6328246183693409
epoch: 2, accuracy: 0.75, loss: 0.6134053282439709
epoch: 3, accuracy: 0.75, loss: 0.5999750606715679
epoch: 4, accuracy: 0.625, loss: 0.5931508243083954
epoch: 5, accuracy: 0.75, loss: 0.5836565047502518
epoch: 6, accuracy: 0.75, loss: 0.5765510965138674
epoch: 7, accuracy: 0.875, loss: 0.5690516084432602
epoch: 8, accuracy: 1.0, loss: 0.5358446128666401
testing accuracy: 1.0 and loss: 0.5067926216870546


In [8]:
train_smt(model, points, labels)
test(model, points, labels, criterion)

weight_0_0 := 1/2
bias_0 := 0.0
weight_0_1 := 0.0
weight_0_2 := 0.0
testing accuracy: 1.0 and loss: 0.5836120843887329
